1. Escolha de idioma

In [4]:
# adicione o código ISO do idioma entre aspas abaixo
language = "pt"

2. Gravação de áudio com Python e JavaScript

In [ ]:
# ao executar a célula, permita acesso ao microfone e grave um áudio de 5 segundos perguntando algo (em caso de erro, cancele e tente novamente)

# bibliotecas a serem utilizadas
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  js_result = output.eval_js("record(%s)" % (sec * 1000))
  audio = b64decode(js_result.split(',')[1])
  file_name = "request_audio.wav"
  with open(file_name, "wb") as f:
    f.write(audio)
  return f"/content/{file_name}"

record_file = record()
display(Audio(record_file, autoplay=True))

3. Reconhecimento de fala com Whisper

In [ ]:
# execute esse bloco para instalar o whisper
!pip install git+https://github.com/openai/whisper.git -q

In [ ]:
# execute a célula para vizualizar e armazenar em uma variável a transcrição de sua fala
import whisper

# para maior precisão, troque abaixo o modelo "small" por "large"
model = whisper.load_model("small")
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

4. Integração com API do Gemini

In [9]:
# execute esse bloco para instalar o gemini
!pip install -q google-generativeai

In [ ]:
# adicione a API_KEY do gemini criada (https://aistudio.google.com/)
# execute a célula para vizualizar e armazenar a resposta do gemini à transcrição de seu áudio anterior (em caso de erro, tente novamente)
import google.generativeai as genai

# adicione a chave abaixo
genai.configure(api_key="API_KEY_AQUI")

model = genai.GenerativeModel("gemma-3-1b-it") # Alterado para gemma-3-1b-it para teste

google_response = model.generate_content(transcription)
print(google_response.text)

5. Sintetizando a resposta como voz (gTTS)

In [ ]:
# execute o bloco para instalar o gTTS
!pip install gTTS

In [ ]:
# execute a célula para escutar em áudio o resultado do
from gtts import gTTS
gtts_object = gTTS(text=google_response.text, lang=language, slow=False)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)

display(Audio(response_audio, autoplay=True))